In [ ]:
# Cell 1: Installations
# !pip install transformers[torch] accelerate -U
!pip install optuna

# Cell 2: Imports
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset
import torch.nn as nn
import shutil
import optuna

# Import from Google Colab's drive module
from google.colab import drive

# Import Hugging Face Transformers components
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers.trainer_callback import EarlyStoppingCallback

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split

# Cell 3: Mount Drive and Define Paths
drive.mount('/content/drive')

# --- Configuration ---
MODEL_NAME = "aubmindlab/bert-base-arabertv2"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

# --- File Paths for Google Drive ---
BASE_DRIVE_DIR = '/content/drive/MyDrive/AraHealthQA/MentalQA/Task1/'
DATA_PATH = os.path.join(BASE_DRIVE_DIR, 'dev_data.tsv')
LABELS_PATH = os.path.join(BASE_DRIVE_DIR, 'train_label.tsv')
TUNING_OUTPUT_DIR = os.path.join(BASE_DRIVE_DIR, 'tuning_output')
FINAL_MODEL_DIR = os.path.join(BASE_DRIVE_DIR, 'final_model')

# Create directories in your Google Drive if they don't exist
os.makedirs(TUNING_OUTPUT_DIR, exist_ok=True)
os.makedirs(FINAL_MODEL_DIR, exist_ok=True)

# Cell 4: Custom Model Definition
class ImprovedMultiLabelModel(nn.Module):
    def __init__(self, model_name, num_labels, alpha=1.0, gamma=2.0):
        super().__init__()
        # Load the pre-trained model, ignoring size mismatches in the classifier layer
        self.bert = AutoModelForSequenceClassification.from_pretrained(
            model_name,
            num_labels=num_labels,
            problem_type="multi_label_classification",
            ignore_mismatched_sizes=True
        )
        self.alpha, self.gamma, self.num_labels = alpha, gamma, num_labels

    def focal_loss(self, logits, labels):
        # A numerically stable implementation of Focal Loss
        BCE_loss = nn.BCEWithLogitsLoss(reduction='none')(logits, labels)
        pt = torch.exp(-BCE_loss)
        return (self.alpha * (1-pt)**self.gamma * BCE_loss).mean()

    def forward(self, input_ids=None, attention_mask=None, labels=None, **kwargs):
        # Get the raw BERT outputs
        outputs = self.bert.bert(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state

        # Use the [CLS] token's representation for classification
        pooled_output = sequence_output[:, 0]
        logits = self.bert.classifier(pooled_output)

        loss = None
        if labels is not None:
            loss = self.focal_loss(logits, labels)

        return SequenceClassifierOutput(loss=loss, logits=logits, hidden_states=outputs.hidden_states, attentions=outputs.attentions)



# Cell 5: Helper Functions
def robust_read_lines(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return [line.strip() for line in f.readlines()]

def load_and_prepare_data(data_path, labels_path):
    questions = robust_read_lines(data_path)
    labels = robust_read_lines(labels_path)
    if len(questions) != len(labels):
        raise ValueError("Mismatch in line count between data and labels.")
    return pd.DataFrame({'text': questions, 'labels_str': labels})

def process_label_strings(label_series):
    return [
        [label.strip() for label in s.split(',') if label.strip()]
        for s in label_series
    ]

def analyze_label_cooccurrence(labels_matrix, label_names):
    cooccurrence_matrix = np.dot(labels_matrix.T, labels_matrix)
    label_frequencies = np.sum(labels_matrix, axis=0)
    cooccurrence_prob = {}
    for i, label1 in enumerate(label_names):
        for j, label2 in enumerate(label_names):
            if i != j and label_frequencies[i] > 0:
                # Calculate P(label2 | label1)
                prob = cooccurrence_matrix[i, j] / label_frequencies[i]
                if prob > 0.3: # Only consider strong correlations
                    cooccurrence_prob[(label1, label2)] = prob
    return cooccurrence_prob

class MentalQADataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)

def adaptive_threshold_prediction(logits, label_names, cooccurrence_prob, base_threshold=0.5):
    probs = 1 / (1 + np.exp(-logits))
    predictions = []
    for i in range(len(probs)):
        sample_probs = probs[i]
        predicted_labels = {label_names[idx] for idx in np.where(sample_probs >= base_threshold)[0]}

        for label in list(predicted_labels):
            for idx, other_label in enumerate(label_names):
                if other_label not in predicted_labels and (label, other_label) in cooccurrence_prob:
                    cooccur_prob = cooccurrence_prob[(label, other_label)]
                    adjusted_threshold = base_threshold * (1 - cooccur_prob * 0.5)
                    if sample_probs[idx] >= adjusted_threshold:
                        predicted_labels.add(other_label)

        if not predicted_labels:
            predicted_labels.add(label_names[np.argmax(sample_probs)])

        if len(predicted_labels) > 4:
            label_prob_pairs = sorted([(label, sample_probs[label_names.index(label)]) for label in predicted_labels], key=lambda x: x[1], reverse=True)
            predicted_labels = {pair[0] for pair in label_prob_pairs[:4]}

        predictions.append(sorted(list(predicted_labels)))
    return predictions


# Cell 6: Main Execution Logic
# Global variables to be set by the initial data load
mlb = None
all_labels = None
cooccurrence_prob = None
train_dataset = None
dev_dataset = None

def objective(trial: optuna.Trial):
    """
    This function defines one trial of the hyperparameter search.
    Optuna will call this function multiple times with different parameter combinations.
    """
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 7e-5, log=True)
    num_train_epochs = trial.suggest_int("num_train_epochs", 5, 20)
    weight_decay = trial.suggest_float("weight_decay", 0.01, 0.1, log=True)
    focal_alpha = trial.suggest_float("focal_alpha", 0.25, 1.5)
    focal_gamma = trial.suggest_float("focal_gamma", 1.0, 3.0)

    # Use a temporary directory inside the main tuning output folder
    trial_output_dir = os.path.join(TUNING_OUTPUT_DIR, f"trial_{trial.number}")

    model = ImprovedMultiLabelModel(
        MODEL_NAME,
        num_labels=len(all_labels),
        alpha=focal_alpha,
        gamma=focal_gamma
    ).to(DEVICE)

    training_args = TrainingArguments(
        output_dir=trial_output_dir,
        num_train_epochs=num_train_epochs,
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        warmup_steps=100,
        logging_strategy="epoch",
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="f1_weighted",
        greater_is_better=True,
        fp16=True, # Always True on Colab GPU
        save_total_limit=1,
    )

    def compute_metrics(p):
        logits, labels = p.predictions, p.label_ids
        base_threshold = trial.suggest_float("base_threshold", 0.2, 0.6)
        predicted_labels_list = adaptive_threshold_prediction(logits, all_labels, cooccurrence_prob, base_threshold=base_threshold)
        y_pred = mlb.transform(predicted_labels_list)
        y_true = labels.astype(int)
        f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)
        return {'f1_weighted': f1}

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=dev_dataset,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )

    trainer.train()
    eval_metrics = trainer.evaluate(eval_dataset=dev_dataset)

    shutil.rmtree(trial_output_dir)
    return eval_metrics['eval_f1_weighted']


def main():
    global mlb, all_labels, cooccurrence_prob, train_dataset, dev_dataset

    print(f"Starting Multi-Label Classification for '{MODEL_NAME}'")
    print("This script will first find the best hyperparameters and then train a final model.")

    print("\n--- 1. Loading and Splitting Data ---")
    try:
        full_df = load_and_prepare_data(DATA_PATH, LABELS_PATH)
    except FileNotFoundError:
        print(f"\nERROR: Data files not found in your Google Drive at {BASE_DRIVE_DIR}")
        return

    train_df, dev_df = train_test_split(full_df, test_size=50, random_state=42, shuffle=True)
    print(f"Using {len(train_df)} samples for training and {len(dev_df)} for development.")

    print("\n--- 2. Preprocessing Labels ---")
    all_labels_flat = [label for sublist in process_label_strings(full_df['labels_str']) for label in sublist]
    all_labels = sorted(list(set(all_labels_flat)))
    print(f"Discovered {len(all_labels)} unique labels: {all_labels}")

    mlb = MultiLabelBinarizer(classes=all_labels)
    mlb.fit(process_label_strings(full_df['labels_str']))
    train_labels = mlb.transform(process_label_strings(train_df['labels_str']))
    dev_labels = mlb.transform(process_label_strings(dev_df['labels_str']))
    cooccurrence_prob = analyze_label_cooccurrence(train_labels, all_labels)
    print(f"Found {len(cooccurrence_prob)} strong label co-occurrence patterns.")

    print("\n--- 3. Tokenizing Text ---")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    train_encodings = tokenizer(train_df['text'].tolist(), truncation=True, padding=True, max_length=256)
    dev_encodings = tokenizer(dev_df['text'].tolist(), truncation=True, padding=True, max_length=256)
    train_dataset = MentalQADataset(train_encodings, train_labels)
    dev_dataset = MentalQADataset(dev_encodings, dev_labels)

    print("\n--- 4. Starting Hyperparameter Optimization with Optuna ---")
    study = optuna.create_study(direction="maximize", study_name="MentalQA_Optimization")
    study.optimize(objective, n_trials=30)

    print("\nOptimization Finished!")
    print(f"Best trial F1 Score: {study.best_value:.4f}")
    print("Best hyperparameters found:")
    best_params = study.best_params
    for key, value in best_params.items():
        print(f"  - {key}: {value}")

    print("\n--- 5. Training Final Model with Best Hyperparameters ---")
    final_model = ImprovedMultiLabelModel(
        MODEL_NAME,
        num_labels=len(all_labels),
        alpha=best_params['focal_alpha'],
        gamma=best_params['focal_gamma']
    ).to(DEVICE)

    final_training_args = TrainingArguments(
        output_dir=FINAL_MODEL_DIR,
        num_train_epochs=best_params['num_train_epochs'],
        learning_rate=best_params['learning_rate'],
        weight_decay=best_params['weight_decay'],
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        warmup_steps=100,
        logging_strategy="epoch",
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="f1_weighted",
        greater_is_better=True,
        fp16=True,
        save_total_limit=1,
    )

    def final_compute_metrics(p):
        logits, labels = p.predictions, p.label_ids
        predicted_labels_list = adaptive_threshold_prediction(logits, all_labels, cooccurrence_prob, base_threshold=best_params['base_threshold'])
        y_pred = mlb.transform(predicted_labels_list)
        y_true = labels.astype(int)
        f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)
        return {'f1_weighted': f1}

    final_trainer = Trainer(
        model=final_model,
        args=final_training_args,
        train_dataset=train_dataset,
        eval_dataset=dev_dataset,
        compute_metrics=final_compute_metrics,
    )

    final_trainer.train()

    print("\n--- 6. Final Performance Analysis on Development Set ---")
    predictions = final_trainer.predict(dev_dataset)
    final_logits = predictions.predictions
    final_predicted_labels_list = adaptive_threshold_prediction(final_logits, all_labels, cooccurrence_prob, base_threshold=best_params['base_threshold'])
    final_predicted_labels_binary = mlb.transform(final_predicted_labels_list)

    final_f1_weighted = f1_score(dev_labels, final_predicted_labels_binary, average='weighted', zero_division=0)
    print(f"\nFinal Weighted F1 Score on dev data: {final_f1_weighted:.4f}")

    print("\n--- Final Per-Label Performance on Dev Set ---")
    print(classification_report(dev_labels, final_predicted_labels_binary, target_names=all_labels, zero_division=0))
    print(f"\nBest model saved in your Google Drive at: {FINAL_MODEL_DIR}")

# Run the main function
if __name__ == '__main__':
    main()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 22.0 MB/s eta 0:00:00
Mounted at /content/drive
Using device: cuda
Starting Multi-Label Classification for 'aubmindlab/bert-base-arabertv2'
This script will first find the best hyperparameters and then train a final model.

--- 1. Loading and Splitting Data ---
Using 300 samples for training and 50 for development.

--- 2. Preprocessing Labels ---
Discovered 7 unique labels: ['A', 'B', 'C', 'D', 'E', 'F', 'Z']
Found 15 strong label co-occurrence patterns.

--- 3. Tokenizing Text ---


tokenizer_config.json:   0%|          | 0.00/611 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...


Optimization Finished!
Best trial F1 Score: 0.6307
Best hyperparameters found:
  - learning_rate: 5.273957732715589e-05
  - num_train_epochs: 13
  - weight_decay: 0.04131058607286182
  - focal_alpha: 0.9702303056621574
  - focal_gamma: 1.39543909126709
  - base_threshold: 0.20408644287720523

--- 5. Training Final Model with Best Hyperparameters ---


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,F1 Weighted
1,0.200200,0.158284,0.578919
2,0.152100,0.170794,0.578919
3,0.138800,0.180320,0.578984
4,0.113700,0.184500,0.591839
5,0.079200,0.223843,0.569782
6,0.051700,0.211459,0.587872
7,0.030400,0.237425,0.579118
8,0.022700,0.236489,0.586392
9,0.015600,0.292072,0.586147
10,0.012400,0.262553,0.592842



--- 6. Final Performance Analysis on Development Set ---



Final Weighted F1 Score on dev data: 0.5928

--- Final Per-Label Performance on Dev Set ---
              precision    recall  f1-score   support

           A       0.67      0.94      0.78        33
           B       0.48      0.92      0.63        24
           C       0.00      0.00      0.00         4
           D       0.50      0.50      0.50        12
           E       0.26      0.90      0.40        10
           F       0.00      0.00      0.00         2
           Z       0.00      0.00      0.00         1

   micro avg       0.48      0.79      0.59        86
   macro avg       0.27      0.47      0.33        86
weighted avg       0.49      0.79      0.59        86
 samples avg       0.50      0.85      0.59        86


Best model saved in your Google Drive at: /content/drive/MyDrive/AraHealthQA/MentalQA/Task1/final_model


In [ ]:
# -*- coding: utf-8 -*-
"""
evaluate_arabert_on_test_set.ipynb

This script loads a fine-tuned AraBERT model from a specific checkpoint
and evaluates its performance on the designated test set.
"""

# Cell 1: Installations
# Ensure necessary libraries are installed in the environment.
# !pip install transformers[torch] accelerate scikit-learn pandas safetensors

# Cell 2: Imports
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset
import torch.nn as nn
from google.colab import drive
from safetensors.torch import load_file

# Import Hugging Face Transformers components
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer
from transformers.modeling_outputs import SequenceClassifierOutput

# Import scikit-learn utilities
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split

# Cell 3: Mount Drive and Define All Paths
print("🗂️ Mounting Google Drive...")
drive.mount('/content/drive')

# --- Configuration ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

# --- Base Paths ---
BASE_DRIVE_DIR = '/content/drive/MyDrive/AraHealthQA/MentalQA/Task1/'

# --- Model Configuration ---
# The original Hugging Face model name used for training
BASE_MODEL_NAME = "aubmindlab/bert-base-arabertv2"
# The specific checkpoint from your training run that you want to evaluate
CHECKPOINT_TO_LOAD = "checkpoint-380"
MODEL_CHECKPOINT_PATH = os.path.join(BASE_DRIVE_DIR, 'final_model', CHECKPOINT_TO_LOAD)
WEIGHTS_PATH = os.path.join(MODEL_CHECKPOINT_PATH, 'model.safetensors') # Using safetensors for secure loading

# --- Data Paths ---
# Use the ORIGINAL training data to fit the binarizer and calculate co-occurrence
TRAIN_DATA_PATH = os.path.join(BASE_DRIVE_DIR, 'dev_data.tsv')
TRAIN_LABELS_PATH = os.path.join(BASE_DRIVE_DIR, 'train_label.tsv')

# The TEST data for final evaluation (150 samples)
TEST_DATA_PATH = os.path.join(BASE_DRIVE_DIR, 'data/subtask1_input_test.tsv')
TEST_LABELS_PATH = os.path.join(BASE_DRIVE_DIR, 'data/subtask1_output_test.tsv')

# --- Output Path ---
# Directory to save the final prediction results
RESULTS_DIR = os.path.join(BASE_DRIVE_DIR, 'results')
os.makedirs(RESULTS_DIR, exist_ok=True)


# Cell 4: Custom Model and Helper Functions (from your training script)
class ImprovedMultiLabelModel(nn.Module):
    """
    The exact same custom model class used during training, including Focal Loss.
    This ensures the architecture matches the saved weights.
    """
    def __init__(self, model_name, num_labels, alpha=1.0, gamma=2.0):
        super().__init__()
        # Load the base model structure
        self.bert = AutoModelForSequenceClassification.from_pretrained(
            model_name,
            num_labels=num_labels,
            problem_type="multi_label_classification",
            ignore_mismatched_sizes=True
        )
        # Store focal loss parameters, though they are not used in evaluation
        self.alpha, self.gamma, self.num_labels = alpha, gamma, num_labels

    def focal_loss(self, logits, labels):
        # This function is not called during prediction but is part of the model definition
        BCE_loss = nn.BCEWithLogitsLoss(reduction='none')(logits, labels)
        pt = torch.exp(-BCE_loss)
        return (self.alpha * (1-pt)**self.gamma * BCE_loss).mean()

    def forward(self, input_ids=None, attention_mask=None, labels=None, **kwargs):
        # Get the raw BERT outputs
        outputs = self.bert.bert(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state

        # Use the [CLS] token's representation for classification
        pooled_output = sequence_output[:, 0]
        logits = self.bert.classifier(pooled_output)

        loss = None
        if labels is not None:
            # Loss calculation is skipped during inference but shown here for completeness
            loss = self.focal_loss(logits, labels)

        return SequenceClassifierOutput(loss=loss, logits=logits, hidden_states=outputs.hidden_states, attentions=outputs.attentions)

def robust_read_lines(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return [line.strip() for line in f.readlines()]

def load_and_prepare_data(data_path, labels_path=None):
    questions = robust_read_lines(data_path)
    if labels_path:
        labels = robust_read_lines(labels_path)
        if len(questions) != len(labels):
            raise ValueError("Mismatch in line count between data and labels.")
        return pd.DataFrame({'text': questions, 'labels_str': labels})
    return pd.DataFrame({'text': questions})

def process_label_strings(label_series):
    return [[label.strip() for label in s.split(',') if label.strip()] for s in label_series]

def analyze_label_cooccurrence(labels_matrix, label_names):
    cooccurrence_matrix = np.dot(labels_matrix.T, labels_matrix)
    label_frequencies = np.sum(labels_matrix, axis=0)
    cooccurrence_prob = {}
    for i, label1 in enumerate(label_names):
        for j, label2 in enumerate(label_names):
            if i != j and label_frequencies[i] > 0:
                prob = cooccurrence_matrix[i, j] / label_frequencies[i]
                if prob > 0.3: # Only consider strong correlations
                    cooccurrence_prob[(label1, label2)] = prob
    return cooccurrence_prob

class MentalQADataset(Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item
    def __len__(self):
        return len(self.encodings['input_ids'])

def adaptive_threshold_prediction(logits, label_names, cooccurrence_prob, base_threshold=0.5):
    probs = 1 / (1 + np.exp(-logits))
    predictions = []
    for i in range(len(probs)):
        sample_probs = probs[i]
        predicted_labels = {label_names[idx] for idx in np.where(sample_probs >= base_threshold)[0]}
        for label in list(predicted_labels):
            for idx, other_label in enumerate(label_names):
                if other_label not in predicted_labels and (label, other_label) in cooccurrence_prob:
                    cooccur_prob = cooccurrence_prob.get((label, other_label), 0)
                    adjusted_threshold = base_threshold * (1 - cooccur_prob * 0.5)
                    if sample_probs[idx] >= adjusted_threshold:
                        predicted_labels.add(other_label)
        if not predicted_labels:
            predicted_labels.add(label_names[np.argmax(sample_probs)])
        if len(predicted_labels) > 4:
            label_prob_pairs = sorted([(l, sample_probs[label_names.index(l)]) for l in predicted_labels], key=lambda x: x[1], reverse=True)
            predicted_labels = {p[0] for p in label_prob_pairs[:4]}
        predictions.append(sorted(list(predicted_labels)))
    return predictions


# Cell 5: Main Evaluation Function
def evaluate_on_test_set():
    """
    Main function to load the fine-tuned AraBERT model and evaluate it on the test set.
    """
    print("🚀 Starting Evaluation of Fine-Tuned AraBERT Model on the Test Set...")

    print(f"\n--- 1. Loading Base Tokenizer and Model from '{BASE_MODEL_NAME}' ---")
    try:
        # Step 1: Load the tokenizer from the original source.
        tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)
        print("✅ Tokenizer loaded successfully.")

        # Step 2: Load the training data to determine the number of labels.
        full_train_df = load_and_prepare_data(TRAIN_DATA_PATH, TRAIN_LABELS_PATH)
        all_labels_flat = [label for sublist in process_label_strings(full_train_df['labels_str']) for label in sublist]
        all_labels = sorted(list(set(all_labels_flat)))
        NUM_LABELS = len(all_labels)
        print(f"Discovered {NUM_LABELS} unique labels from training data.")

        # Step 3: Instantiate the model architecture.
        print("Instantiating model architecture...")
        model = ImprovedMultiLabelModel(model_name=BASE_MODEL_NAME, num_labels=NUM_LABELS)

        # Step 4: Load the fine-tuned weights from your specific checkpoint.
        print(f"Loading fine-tuned weights from: {WEIGHTS_PATH}")
        if not os.path.exists(WEIGHTS_PATH):
            raise FileNotFoundError(f"Weights file not found at {WEIGHTS_PATH}. Please ensure the checkpoint path is correct.")
        state_dict = load_file(WEIGHTS_PATH, device=DEVICE.type)

        # Step 5: Apply the loaded weights to the model structure.
        model.load_state_dict(state_dict)
        model.to(DEVICE)
        print("✅ Model architecture created and fine-tuned weights applied successfully.")

    except Exception as e:
        print(f"❌ FATAL ERROR during model loading: {e}")
        return

    print("\n--- 2. Preprocessing Labels and Co-occurrence from Training Data ---")
    mlb = MultiLabelBinarizer(classes=all_labels).fit(process_label_strings(full_train_df['labels_str']))

    # We use a train/dev split of the original data to calculate co-occurrence, just like in training.
    train_df, _ = train_test_split(full_train_df, test_size=50, random_state=42, shuffle=True)
    train_labels_binary = mlb.transform(process_label_strings(train_df['labels_str']))
    cooccurrence_prob = analyze_label_cooccurrence(train_labels_binary, all_labels)
    print(f"Calculated {len(cooccurrence_prob)} strong label co-occurrence patterns.")

    print("\n--- 3. Loading and Tokenizing Test Data ---")
    test_df = load_and_prepare_data(TEST_DATA_PATH, TEST_LABELS_PATH)
    print(f"Loaded {len(test_df)} samples from the test set.")
    test_encodings = tokenizer(test_df['text'].tolist(), truncation=True, padding=True, max_length=256)
    test_dataset = MentalQADataset(test_encodings)

    print("\n--- 4. Generating Predictions for the Test Set ---")
    # A basic Trainer is used here just as a predictor. No training arguments are needed.
    trainer = Trainer(model=model)
    raw_predictions = trainer.predict(test_dataset)
    logits = raw_predictions.predictions

    print("\n--- 5. Post-processing Predictions with Adaptive Thresholding ---")
    # ⚠️ IMPORTANT: Replace this with the 'base_threshold' from your Optuna study results!
    best_base_threshold = 0.45 # <--- REPLACE THIS VALUE
    print(f"Using the best base_threshold found during tuning: {best_base_threshold:.4f}")
    predicted_labels_list = adaptive_threshold_prediction(logits, all_labels, cooccurrence_prob, base_threshold=best_base_threshold)

    print("\n--- 6. Final Evaluation on the Test Set ---")
    y_true_binary = mlb.transform(process_label_strings(test_df['labels_str']))
    y_pred_binary = mlb.transform(predicted_labels_list)
    weighted_f1 = f1_score(y_true_binary, y_pred_binary, average='weighted', zero_division=0)

    print("\n--- 🥁 Final Test Set Results 🥁 ---")
    print(f"Weighted F1 Score: {weighted_f1:.4f}")
    print("------------------------------------\n")
    print("--- Per-Label Performance (Test Set) ---")
    print(classification_report(y_true_binary, y_pred_binary, target_names=all_labels, zero_division=0))

    print("\n--- 7. Saving Predictions to File ---")
    test_df['Predicted_Labels'] = [",".join(p) for p in predicted_labels_list]
    prediction_output_path = os.path.join(RESULTS_DIR, f"arabert_{CHECKPOINT_TO_LOAD}_test_predictions.tsv")
    test_df[['Predicted_Labels']].to_csv(prediction_output_path, sep='\t', header=False, index=False)
    print(f"💾 Test set predictions saved to: {prediction_output_path}")

    print("\n✅ Evaluation complete.")


# Cell 6: Run the Evaluation
if __name__ == '__main__':
    evaluate_on_test_set()

🗂️ Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Using device: cuda
🚀 Starting Evaluation of Fine-Tuned AraBERT Model on the Test Set...

--- 1. Loading Base Tokenizer and Model from 'aubmindlab/bert-base-arabertv2' ---


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Tokenizer loaded successfully.
Discovered 7 unique labels from training data.
Instantiating model architecture...
Loading fine-tuned weights from: /content/drive/MyDrive/AraHealthQA/MentalQA/Task1/final_model/checkpoint-380/model.safetensors
✅ Model architecture created and fine-tuned weights applied successfully.

--- 2. Preprocessing Labels and Co-occurrence from Training Data ---
Calculated 15 strong label co-occurrence patterns.

--- 3. Loading and Tokenizing Test Data ---
Loaded 150 samples from the test set.

--- 4. Generating Predictions for the Test Set ---



--- 5. Post-processing Predictions with Adaptive Thresholding ---
Using the best base_threshold found during tuning: 0.4500

--- 6. Final Evaluation on the Test Set ---

--- 🥁 Final Test Set Results 🥁 ---
Weighted F1 Score: 0.5429
------------------------------------

--- Per-Label Performance (Test Set) ---
              precision    recall  f1-score   support

           A       0.65      0.81      0.72        84
           B       0.60      0.75      0.67        85
           C       0.00      0.00      0.00        10
           D       0.37      0.21      0.26        34
           E       0.41      0.37      0.39        38
           F       0.00      0.00      0.00         6
           Z       0.00      0.00      0.00         3

   micro avg       0.58      0.59      0.58       260
   macro avg       0.29      0.31      0.29       260
weighted avg       0.51      0.59      0.54       260
 samples avg       0.65      0.65      0.60       260


--- 7. Saving Predictions to File ---